<a href="https://colab.research.google.com/github/Jersae/Google-TimeSeries-workshop/blob/main/TSSD_TF2_Human_Activity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -qq https://www.dropbox.com/s/ecs4lywacmbiw6t/HAR_Smartphones.zip
  
!unzip -qq HAR_Smartphones.zip
!rm HAR_Smartphones.zip
!rm -r __MACOSX
!ls

 sample_data  'UCI HAR Dataset'


In [ ]:
!ls 'UCI HAR Dataset'

activity_labels.txt  features_info.txt	features.txt  README.txt  test	train


In [ ]:
# lstm model for the har dataset
%matplotlib inline
import matplotlib.pyplot as plt

import os
from datetime import datetime

import numpy as np
import pandas as pd

import tensorflow as tf


from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Input

from tensorflow.keras.utils import to_categorical


In [ ]:
print(tf.__version__)

2.4.1


## Load the data

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset/')
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	return trainX, trainy, testX, testy

In [ ]:
# load data
X_train, y_train, X_test, y_test = load_dataset()

## Model

In [ ]:
batch_size = 64
epochs = 25

n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]


In [ ]:

Inp = Input(shape=(n_timesteps, n_features))
# x = LSTM(100, activation='relu', name= 'LSTM_01',return_sequences=True)(Inp)
x = LSTM(100, activation='relu', name= 'LSTM_01')(Inp)
# x = Dropout(0.5)(x)
x = Dense(100,activation='relu', name= 'Dense_01')(x)
out = Dense(n_outputs, activation='softmax')(x)


print('timesteps:', n_timesteps, ' features: ', n_features, ' Classes:',n_outputs)

timesteps: 128  features:  9  Classes: 6


In [ ]:
model = Model(Inp,out)

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 9)]          0         
_________________________________________________________________
LSTM_01 (LSTM)               (None, 100)               44000     
_________________________________________________________________
Dense_01 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense (Dense)                (None, 6)                 606       
Total params: 54,706
Trainable params: 54,706
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit network
model.fit(X_train, y_train, 
          epochs=epochs, 
          batch_size=batch_size, 
          validation_data=(X_test,y_test),
          verbose=1)


Epoch 1/25
115/115 [==============================] - 21s 164ms/step - loss: 4385695432.4499 - accuracy: 0.4069 - val_loss: 1.4412 - val_accuracy: 0.5348
Epoch 2/25
115/115 [==============================] - 18s 160ms/step - loss: 1.3484 - accuracy: 0.5527 - val_loss: 1.2430 - val_accuracy: 0.5779
Epoch 3/25
115/115 [==============================] - 18s 160ms/step - loss: 1.1650 - accuracy: 0.6198 - val_loss: 1.1096 - val_accuracy: 0.6016
Epoch 4/25
115/115 [==============================] - 18s 158ms/step - loss: 1.0330 - accuracy: 0.6311 - val_loss: 1.0253 - val_accuracy: 0.5816
Epoch 5/25
115/115 [==============================] - 18s 155ms/step - loss: 0.9407 - accuracy: 0.6598 - val_loss: 0.9607 - val_accuracy: 0.6101
Epoch 6/25
115/115 [==============================] - 18s 159ms/step - loss: 0.8839 - accuracy: 0.6684 - val_loss: 0.9152 - val_accuracy: 0.6240
Epoch 7/25
115/115 [==============================] - 18s 158ms/step - loss: 0.8394 - accuracy: 0.6750 - val_loss: 0.8834

In [ ]:
# evaluate model
model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)


[0.7494626641273499, 0.7037665247917175]

In [ ]:
n_outputs

6